In [1]:
#Import Libraries necessary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

C:\Users\sosin\AppData\Local\Temp\ipykernel_11408\921819559.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Open file called combined and drop the first column as its a duplicate
df = pd.read_csv("Combined.csv")
firstColumn = df.columns[0]
df = df.drop([firstColumn], axis=1)
df.head(5)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111259 entries, 0 to 1111258
Data columns (total 47 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   flow_duration    1111259 non-null  float64
 1   Header_Length    1111259 non-null  float64
 2   Protocol Type    1111259 non-null  float64
 3   Duration         1111259 non-null  float64
 4   Rate             1111259 non-null  float64
 5   Srate            1111259 non-null  float64
 6   Drate            1111259 non-null  float64
 7   fin_flag_number  1111259 non-null  float64
 8   syn_flag_number  1111259 non-null  float64
 9   rst_flag_number  1111259 non-null  float64
 10  psh_flag_number  1111259 non-null  float64
 11  ack_flag_number  1111259 non-null  float64
 12  ece_flag_number  1111259 non-null  float64
 13  cwr_flag_number  1111259 non-null  float64
 14  ack_count        1111259 non-null  float64
 15  syn_count        1111259 non-null  float64
 16  fin_count        1

In [3]:
#Replace Dictionary attack as 1 and beningn as 0 and show the data
df["label"].replace(["DictionaryBruteForce","BenignTraffic"], [1,0], inplace=True)

df.head(5)
df.info()


C:\Users\sosin\AppData\Local\Temp\ipykernel_11408\2211177348.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["label"].replace(["DictionaryBruteForce","BenignTraffic"], [1,0], inplace=True)
C:\Users\sosin\AppData\Local\Temp\ipykernel_11408\2211177348.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111259 entries, 0 to 1111258
Data columns (total 47 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   flow_duration    1111259 non-null  float64
 1   Header_Length    1111259 non-null  float64
 2   Protocol Type    1111259 non-null  float64
 3   Duration         1111259 non-null  float64
 4   Rate             1111259 non-null  float64
 5   Srate            1111259 non-null  float64
 6   Drate            1111259 non-null  float64
 7   fin_flag_number  1111259 non-null  float64
 8   syn_flag_number  1111259 non-null  float64
 9   rst_flag_number  1111259 non-null  float64
 10  psh_flag_number  1111259 non-null  float64
 11  ack_flag_number  1111259 non-null  float64
 12  ece_flag_number  1111259 non-null  float64
 13  cwr_flag_number  1111259 non-null  float64
 14  ack_count        1111259 non-null  float64
 15  syn_count        1111259 non-null  float64
 16  fin_count        1

In [4]:
#X is the independant variable and Y is the dependant 
X = df.iloc[:,0:46] 
Y = df.iloc[:,-1] 

print(Y)

0          0
1          0
2          0
3          0
4          0
          ..
1111254    1
1111255    1
1111256    1
1111257    1
1111258    1
Name: label, Length: 1111259, dtype: int64


In [5]:
#This piece of code is used to find the best features and show the score of the features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestFeatures = SelectKBest(score_func=chi2, k=3)
fit = bestFeatures.fit(X,Y)

dfScores = pd.DataFrame(fit.scores_)#
dfColumns = pd.DataFrame(X.columns)

featuresScores = pd.concat([dfColumns, dfScores], axis=1)

featuresScores.columns = ["Features", "Score"]

featuresScores.sort_values(by="Score")

,Features,Score
40,Number,2.521993e-03
31,IPv,9.898868e-02
32,LLC,9.898868e-02
30,ICMP,1.189588e-01
45,Weight,1.519541e-01
9,rst_flag_number,1.794239e-01
21,DNS,5.164218e+00
29,ARP,2.698145e+01
44,Variance,4.568859e+01
16,fin_count,1.101825e+02


In [6]:
#Features which were decided to be the best
X = df[["flow_duration", "Header_Length", "Protocol Type", "Duration","Rate" ]] 

In [7]:
#data splitting into train and test for both X and Y with the test size of 20% as is the standard 
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=25)


In [8]:
#Creation of the XGB model with default values and binary classification
xgb_model = XGBClassifier(
    objective='binary:logistic'  # for binary classification     
)

In [9]:
#fit the train data into the model
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_val)

In [10]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

# Additional evaluation metrics
print(classification_report(y_val, y_pred))

Accuracy: 0.989808865612008
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    219593
           1       0.75      0.22      0.34      2659

    accuracy                           0.99    222252
   macro avg       0.87      0.61      0.67    222252
weighted avg       0.99      0.99      0.99    222252



In [11]:
from sklearn.metrics import recall_score

# getting recall score of the model
ensemble_recall = recall_score(y_val, y_pred)

print("Stacking Ensemble Model Recall:", ensemble_recall)

Stacking Ensemble Model Recall: 0.2237683339601354


In [12]:
from sklearn.metrics import f1_score

#getting f1 score of the model
ensemble_f1_score = f1_score(y_val, y_pred)

print("Stacking Ensemble Model F1 Score:", ensemble_f1_score)

Stacking Ensemble Model F1 Score: 0.34442836468885674
